In [8]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
import numpy as np

In [9]:
X_train = np.load("../../../Processing/data/X_train.npy")
y_train = np.load("../../../Processing/data/y_train.npy")
X_test = np.load("../../../Processing/data/X_test.npy")
y_test = np.load("../../../Processing/data/y_test.npy")
real_test = np.load("../../../Processing/data/real_test.npy")
targets = np.load("../../../Processing/data/targets.npy")
all_train = np.load("../../../Processing/data/all_train.npy")

In [10]:
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
parameter_grid = {
                  "C":[1]
                  }
cross_validation = StratifiedKFold(y_train, n_folds=5)
gs = GridSearchCV(estimator = SVC(kernel="linear"),
                   param_grid = parameter_grid,
                  cv = cross_validation)

In [11]:
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.824438202247191
{'C': 1}


In [12]:
clf = gs.best_estimator_
clf.fit(X_train, y_train)
print('Test accuracy: %.3f' % clf.score(X_test, y_test))

Test accuracy: 0.788


全データをfitさせてテストデータに対する出力を求める

In [13]:
clf.fit(all_train, targets)
test = pd.read_csv("../../../Training_test_data/test.csv")
output = clf.predict(real_test).astype(int)
df_output = pd.DataFrame()
df_output['PassengerId'] = test['PassengerId']
df_output['Survived'] = output
df_output[['PassengerId','Survived']].to_csv('../../../Submission/svm_only_output.csv',index=False)

Kernel SVM

In [14]:
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

best_score = 0.857
for i in range(5,20):
    i = i / 100    
    for j in range(100,250):
        j = j / 100
        clf_svm = SVC(kernel='linear',random_state=0,gamma=i,C=j)
        pipe_svm = Pipeline([['sc', StandardScaler()],['clf', clf_svm]])
        scores = cross_val_score(estimator=clf_svm,
                                 X=X_train,
                                 y=y_train,
                                 cv=10,
                                 scoring='roc_auc')
        if scores.mean() > best_score:
            print("gamma:",i,"C:",j,scores.mean(),"+-",scores.std())


In [15]:
clf = SVC(kernel='rbf',random_state=0,gamma=0.06,C=1.01)

In [16]:
clf.fit(X_train, y_train)
print('Test accuracy: %.3f' % clf.score(X_test, y_test))

Test accuracy: 0.827


In [17]:
clf.fit(all_train, targets)
test = pd.read_csv("../../../Training_test_data/test.csv")
output = clf.predict(real_test).astype(int)
df_output = pd.DataFrame()
df_output['PassengerId'] = test['PassengerId']
df_output['Survived'] = output
df_output[['PassengerId','Survived']].to_csv('../../../Submission/kernel_svm_only_output.csv',index=False)